### Web scrape valid surf rides from the Smartfin website: 

In [15]:
#Imports 
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import time

print("Done.")

Done.


### Scrape verified surf sessions into a dataframe (can add more later):

In [16]:
# Start by just looking at my surf sessions in SD (near CDIP buoy): 

# 15218 - First VIRB filmed session - Oct. 24, 2018
# 15669 - Second VIRB filmed session - Nov. 7, 2018
# 15692 - Third VIRB filmed session - Nov. 9, 2018
# 15686 - Fourth VIRB filmed session - Nov. 11, 2018

ride_ids = [15218, 15669, 15692, 15686]

In [17]:
#Get urls for surf sessions in Scripps beach area that most likely correspond to surfing time intervals: 
url_45_60 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=45&durationMax=60&sensorTypeWave=true#searchResultsContainer'
url_61_80 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=61&durationMax=80&sensorTypeWave=true#searchResultsContainer'
url_81_100 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=81&durationMax=100&sensorTypeWave=true#searchResultsContainer'
url_101_130 = 'https://surf.smartfin.org/advanced_search/?northEastLat=32.6&northEastLon=-117.24&southWestLat=32.9&southWestLon=-117.33&dateTimeBegin=&dateTimeEnd=&timeZone=PDT&durationMin=101&durationMax=130&sensorTypeWave=true#searchResultsContainer'

urls = [url_61_80]
#urls = [url_45_60, url_61_80, url_81_100, url_101_130]

In [18]:
# Define a function that will parse rows from data tables on CDIP's website:
def parse_rows(row_number):
    name_string = ""
    name_list = []
    for t in tr_elements[row_number]:
        name = t.text_content()
        for i in name: 
            name_string += i
    #print(name_string)

    # Create a string from the values
    name_string = name_string.split(" ")
    
    # Remove all spaces from the list
    for i in name_string: 
        if len(i) > 0:
            name_list.append(i)
      
    # Ensure that time stays with 'Date (UTC)'' header
    name_list[0] = name_list[0] + " " +  name_list[1]
    name_list.pop(1)
    
    return name_list

In [19]:
# Iterate over each url and get all ride_ids located at that url: 
ride_ids = []
for u in urls: 
    
    # Create a handle, page, to handle the contents of the website
    page = requests.get(u)

    # Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    # Parse data stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    # Retrieve all of the ride ids: 
    for j in range(1, len(tr_elements)):
        data = parse_rows(j)
        data = str(data[0]).strip("\n")
        if int(data) > 10000: 
            ride_ids.append(data)

print(len(ride_ids))
print(ride_ids[:20])
print(ride_ids[-10:])

265
[' 16302', ' 16301', ' 16298', ' 16290', ' 16262', ' 16243', ' 16237', ' 16236', ' 16196', ' 16195', ' 16192', ' 16168', ' 16161', ' 16148', ' 16073', ' 16072', ' 16000', ' 15972', ' 15947', ' 15859']
[' 10191', ' 10157', ' 10085', ' 10084', ' 10068', ' 10060', ' 10040', ' 10036', ' 10015', ' 10008']


In [20]:
# Fin ID Scraper: 

ride_url_base = 'http://surf.smartfin.org/ride/'
str_id_csv = 'img id="temperatureChart" class="chart" src="' # Look for this text in the HTML contents in fcn below

def read_csv_from_ride2(ride): 
    smartfin_url_base = 'http://surf.smartfin.org'
    ride_url = ride_url_base + str(ride)
    html_page = requests.get(ride_url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    #print(soup)
    links = []
    for link in soup.findAll('a'):
        links.append(link)
    if len(links) > 8: 
        motion_csv_link = str(links[8])
        #print(links[8])
        motion_csv_link = motion_csv_link.split(" ")
        motion_csv_link = motion_csv_link[1]
        motion_csv_link = motion_csv_link.split("\"")
        motion_csv_link = motion_csv_link[1]

        #print(motion_csv_link)

        motion_csv_link = smartfin_url_base + motion_csv_link
        motion_df_small = pd.read_csv(motion_csv_link, parse_dates = [0])

        return 1, motion_df_small
    
    else: 
        
        return 0, 0

In [21]:
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

counter = 0
index = 1
for ride in ride_ids[10:30]: 
    #ride = '16167'
    b, motion_df = read_csv_from_ride2(ride)
    # Check to make sure that a .CSV link was found for motion: 
    if b == 1: 
        counter += 1
        print(counter, len(motion_df))
        motion_df.set_index('UTC', drop = True, append = False, inplace = True)
        appended_multiIndex.append(str(ride)) # build list to be multiIndex of future DataFrame
        appended_motion_list.append(motion_df)
    print(index)
    index += 1

print("Total number of rides downloaded: ", counter)
df_keys = tuple(appended_multiIndex)
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])

    

1
2
3
4
5
6
7
1 20621
8
9
10
11
12
13
14
15
16
17
18
19
20
Total number of rides downloaded:  1


In [22]:
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

counter = 0
index = 1
for ride in ride_ids[30:60]: 
    #ride = '16167'
    b, motion_df = read_csv_from_ride2(ride)
    # Check to make sure that a .CSV link was found for motion: 
    if b == 1: 
        counter += 1
        print(counter, len(motion_df))
        motion_df.set_index('UTC', drop = True, append = False, inplace = True)
        appended_multiIndex.append(str(ride)) # build list to be multiIndex of future DataFrame
        appended_motion_list.append(motion_df)
    print(index)
    index += 1

print("Total number of rides downloaded: ", counter)
df_keys = tuple(appended_multiIndex)
motion_df2 = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Total number of rides downloaded:  0


ValueError: No objects to concatenate

In [ ]:
motion_df2[:10]

In [ ]:
'''
appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

print("Once the counter gets to ", len(ride_ids), " it will be done printing.")

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0
    
# Loop over ride_ids and find CSVs
counter = 0
for rid in ride_ids:
    counter += 1
    if counter % 10 == 0:
        print(counter)
    try:
        new_motion_df = read_csv_from_ride(rid) # get given ride's CSV from its ride ID using function above
        new_motion_df.set_index('UTC', drop = True, append = False, inplace = True)            

        if not new_motion_df.empty: # Calibration rides, for example
            # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
            appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
            #appended_ocean_list.append(new_ocean_df)
            appended_motion_list.append(new_motion_df)
            #print("Ride data has been uploaded.")
            #print("Ride: ", rid, "data has been uploaded.")
            count_good_fins += 1
            #print("Ride ", rid, "worked!")    
     
    except: 
        print("Ride ", rid, "threw an exception!")    

#%% Build the "Master" DataFrame

# appended_ocean_df.summary()

df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed
#ocean_df = pd.concat(appended_ocean_list, keys = df_keys, names=['ride_id'])
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])


##Here, maybe just use info from the motion_df and don't worry about ocean_df data for now.
##If you do want ocean_df data, look at how Phil was getting it from "July 10th and 11th Calibration" jupyter notebook file.
#print(motion_df)
#print(appended_motion_list)
print("Done.")
'''

In [ ]:
# View data in a python dataframe: 
print(len(motion_df))
motion_df[0:10]

# I think that we're resampling each surf session at too small of a rate? 
# Why do we have so little data? 

#Sampling interval of 33ms: 3,290,682 initial data points, 438,099 final data points (adding .033 to UTC time)
#Sampling interval of 20ms: 5,429,605 initial data points, 438,099 final data points (adding .020 to UTC time)

In [ ]:
motion_df.shape

In [ ]:
# Drop latitude/longitude columns and remove NaN rows: 
motion_df = motion_df.drop(["Latitude", "Longitude"], axis=1)
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:20]

### Scrape data from same-day CDIP and add it as an additional column to the dataframe:


In [ ]:
# Format should be appending year, month
# Example: Sept. 2019 -> 201909
def create_url_string(year, month):
    url='https://cdip.ucsd.edu/themes/cdip?tz=UTC&numcolorbands=10&palette=cdip_classic&zoom=auto&ll_fmt=dm&high=6.096&r=999&un=1&pb=1&d2=p70&u2=s:201:st:1:v:parameter:dt:'
    if 2014 <= int(year) <= 2019:
        year = year
    if 1 <= int(month) <= 12 and len(month) == 2:
        month = month
    url += year
    url += month
    return url 

In [ ]:
# For each row in the dataframe, parse the index ('UTC' column) to get the year, month information. 
motion_df.columns

dates = []
times = []
for row in motion_df.index:
    time = str(row[1])
    date = time.split(" ")
    time = date[1]
    date = date[0]
    time = time.split(".")
    time = time[0]
    time = time.split(":")
    time = time[0] + ":" + time[1]
    times.append(time)
    dates.append(date)
print(dates[0:10], times[0:10])

motion_df["Date"] = dates
motion_df["Time"] = times

In [ ]:
motion_df[0:10]

In [ ]:
# Define a function that will find the closest time on the CDIP table to the surf ride's time: 
def find_closest_single_time(date, time_utc):
    time_list = []
    
    # Iterate over each date
    for x in df_data['Date(UTC)']:
        a = x.split(" ")
        if date == a[0]:
            a = str(a[1]).split(':')
            y = str(time_utc).split(":")
            #print(a)
            #print(y)
            
            # Find the closest timestamp (less than 15 minutes away)
            time1 = int(a[0])*60 + int(a[1])
            time2 = int(y[0])*60 + int(y[1])
            
            if abs(time1 - time2) <= 15:
                time_string = str(a[0]) + ":" + str(a[1])
                time_list.append(time_string)
           
    return time_list

### Create the output labels from CDIP data for each Smartfin timestamp: 


In [ ]:
# Create the output labels from CDIP data for each Smartfin timestamp: 
old_url = 'old_url'
old_date = 'old_date'
old_time = 'xx:xx'

wave_heights = []
wave_periods = []
wave_directions = []
for d, time in zip(dates,times):
    di = d.split("-")
    year = di[0]
    month = di[1]
    url = create_url_string(year, month)
    
    #We need to load a new table since we have a new surf ride session:
    if (url != old_url): 
        old_url = url
        # Create a handle, page, to handle the contents of the website
        page = requests.get(url)

        # Store the contents of the website under doc
        doc = lh.fromstring(page.content)

        # Parse data stored between <tr>..</tr> of HTML
        tr_elements = doc.xpath('//tr')

        # Need to drop the first 3 rows since they aren't in the table
        tr_elements = tr_elements[3:]
        
        # Parse the first row as the header
        tr_elements = doc.xpath('//tr')

        # Create empty list
        headers = []
        i = 0

        # For each row, store each first element (header) and an empty list
        for t in tr_elements[3]:
            i+=1
            name=t.text_content()
            #print(name)
            headers.append(name)

        #print(headers)

        # Create a Pandas dataframe: 
        data_list = []

        #Since out first row is the header, data is stored on the second row onwards
        for j in range(4, len(tr_elements)):
            data = parse_rows(j)
            data_list.append(data)

        df_data = pd.DataFrame(data_list, columns=headers)
        #print(df_data[:10])
        
        #Find the closest time in the CDIP data table that corresponds to the Smartfin data: 
        date = d #YYYY-MM-DD format
        print("date, time ", date, time)
        time_list = find_closest_single_time(date, time)
        print("time list ", time_list)
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    print("height ", df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction) 
            
    # Same date/month but maybe a different day or time:
    elif old_date != date or old_time[3] != time[3]:
        date = d #YYYY-MM-DD format
        #print(date, time)
        time_list = find_closest_single_time(date, time)
        #print(time_list)
        
        old_date = date
        old_time = time
        
        # Now compute the average significant wave height for that time period:
        length = df_data['Date(UTC)'].size
        wave_height_list = []
        wave_period_list = []
        wave_direction_list = []
        for time in time_list: 
            date_data = date + " " + time
            for i in range(0, length):
                if df_data['Date(UTC)'][i] == date_data:
                    #print(df_data['Hs(ft)'][i])
                    wave_height_list.append(float(df_data['Hs(ft)'][i]))
                    wave_period_list.append(float(df_data['Tp(s)'][i]))
                    wave_direction_list.append(float(df_data['Dp(deg)'][i]))
                    
      
        # Throws error when date doesn't exist on CDIP data (ex: 2018-01-18)
        if len(wave_height_list) == 0 or len(wave_period_list) == 0 or len(wave_direction_list) == 0: 
            avg_wave_height = np.nan
            avg_wave_period = np.nan
            avg_wave_direction = np.nan
            
        else: 
            #print(wave_height_list)
            avg_wave_height = sum(wave_height_list)/len(wave_height_list)
            avg_wave_period = sum(wave_period_list)/len(wave_period_list)
            avg_wave_direction = sum(wave_direction_list)/len(wave_direction_list)
            #print(avg_wave_height)
            #print(avg_wave_period)
            #print(avg_wave_direction)    

    # If they have the same url, date, and time then they will have the same value computed. 
    wave_heights.append(avg_wave_height)
    wave_periods.append(avg_wave_period)
    wave_directions.append(avg_wave_direction)

In [ ]:
print(len(dates))
print(len(wave_heights))
print(len(wave_periods))
print(len(wave_directions))


# Check to make sure different wave heights were appended throughout the dataframe: 
motion_df["Hs(ft)"] = wave_heights
motion_df["Tp(s)"] = wave_periods
motion_df["Dp(deg)"] = wave_directions

for x in range(0, 60000, 10000):
    print(motion_df.iloc[[x]]['Hs(ft)'])
    print(motion_df.iloc[[x]]['Tp(s)'])
    print(motion_df.iloc[[x]]['Dp(deg)'])

motion_df[0:5]

In [ ]:
# Now drop all of the nan values that I introduced earlier: 
motion_df = motion_df.dropna(axis=0, how='any')
print(len(motion_df))
motion_df[0:5]

In [ ]:
# We're collecting IMU data at 3-4 Hz here. 
# We're using 4 surf sessions and we have 60,000 data points. 
# If we only calculated Hs once per minute then we would have 330 data points. 

# Instead of looking at significant wave height, we could look at wave direction 
# or we could look at Hs, dir, and period and see which one we predict best. 



In [ ]:
# Convert raw IMU values to real values: 
g_const = 512 #Raw acceleration constant 512 = 1g (accelerometer's measured force due to gravity)
gravity = 9.80665 #Approximate measurement for gravity

# Correct the IMU Acceleration columns into units of meters
# Dividing by 512 is equivalent to muliplying by 4 to correct the bit shifting by 2 places and dividing by 2048 to convert bits to G's
# Multiplying by the 9.81 afterwards is simply to convert G's into m/s^2
motion_df['IMU A1'] = motion_df['IMU A1'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A2'] = motion_df['IMU A2'].apply(lambda x: x / g_const * gravity)
motion_df['IMU A3'] = motion_df['IMU A3'].apply(lambda x: x / g_const * gravity)

In [ ]:
# Gyroscopic Rotation converts to deg/s
gyro_const = 8.2 # Raw gyrscope constant 8.2 bits per degree
motion_df['IMU G1'] = motion_df['IMU G1'].apply(lambda x: x / gyro_const)
motion_df['IMU G2'] = motion_df['IMU G2'].apply(lambda x: x / gyro_const)
motion_df['IMU G3'] = motion_df['IMU G3'].apply(lambda x: x / gyro_const)

In [ ]:
# Magnetometer values
# Offset variables help in recentering the magnetic data in order to define direction and use trig functions
'''
M1_offset_var = 219.786
M2_offset_var = 180
M3_offset_var = 280

motion_df['IMU M1'] = motion_df['IMU M1'].apply(lambda x: x - M1_offset_var)
motion_df['IMU M2'] = motion_df['IMU M2'].apply(lambda x: x - M2_offset_var)
motion_df['IMU M3'] = motion_df['IMU M3'].apply(lambda x: x - M3_offset_var)
'''

In [ ]:
print(len(motion_df))
motion_df[:20]

In [ ]:
# After converting all to real values, try to export data to .CSV so everyone else doesn't have to webscrape it:
#motion_df.to_csv('CSE258_A2_Data.csv')

In [ ]:
#motion_df.to_csv('CSE258_A2_Data2.csv')